# 1. 타겟종이 존재하는 위치 주변의 정보 파악하기  

In [1]:
import pandas as pd
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
from haversine import haversine

## 1-1. 타겟종 주변의 무당벌레 종 분포 파악

In [2]:
# 무당벌레 전체 관측목록
kellog = pd.read_csv("./kellog_location.csv")

In [3]:
gps =[]
for i in kellog.gps:
    gps.append((float(i.split(',')[0].replace("(","")),float(i.split(',')[1].replace(")",""))))

In [4]:
kellog.gps = gps

In [5]:
# 무당벌레 전체 관측목록
df = pd.read_csv("./Data/20220209_07_21_무당벌레관측.csv")



# gps 오차 1000m미만만 남기기
df = df[df.public_positional_accuracy<=1000].reset_index(drop=True)


#Latitude, Longitude 0으로 기록된 부분들 삭제
df = df[(df.Latitude!=0.0)&(df.Longitude!=0.0)].reset_index(drop=True)


#gps 변환
df.Latitude = df.Latitude.astype(float)
df.Longitude = df.Longitude.astype(float)

df_gps = []
for i in tqdm(range(len(df))):
    df_gps.append( (df.Latitude[i], df.Longitude[i]) )
    
    
df['gps'] = df_gps   

#sp.로 들어간 애들을 제거
sp_idx = []
for i in range(len(df)):
    if 'sp.' in df.Species[i]:
        sp_idx.append(i)
df= df.drop(sp_idx).reset_index(drop=True)

# value_counts 해서 통계적으로 유의미한 30개도 안되는 애들 고르기
under_30 = []
for i in range(len(df.Species.value_counts())):
    if df.Species.value_counts()[i]<30:
        under_30.append(df.Species.value_counts().index[i])
        
        
# 고른애들과 이름이 같다면 drop해야함
drop_30 = []
for i in tqdm(range(len(df.Species))):
    for j in under_30:
        if df.Species[i] == j:
            drop_30.append(i)        
df= df.drop(drop_30).reset_index(drop=True)            
            
# 속만 있는 경우 제거
one_letter = []
for i in df.Species.value_counts().index:
    if len(i.split(' ')) == 1:
        one_letter.append(i)    
        
drop_one = []
for i in tqdm(range(len(df.Species))):
    for j in one_letter:
        if df.Species[i] == j:
            drop_one.append(i)        
df = df.drop(drop_one).reset_index(drop=True)


# 타겟종이 될 종만 남긴 데이터 프레임 생성
target = 'Coccinella novemnotata'
df_target = df[df.Species==target].reset_index(drop=True)



# 타겟종의 데이터프레임에 주변 정보를 카운트해서 넣을 컬럼 생성
for i in df.Species.value_counts().index:

    df_target[i+'_18'] = 0

    
    


100%|███████████████████████████████████████████████████████████████████████| 175834/175834 [00:13<00:00, 13214.80it/s]


In [6]:
# 타겟종의 데이터프레임에 주변 정보를 카운트해서 넣을 컬럼 생성
for i in df.Species.value_counts().index:

    kellog[i+'_18'] = 0

In [7]:
# 주변 종 체크
for i in tqdm(range(len(kellog))): # target만 뽑아놓은 데이터 프레임
    for j in range(len(df.gps)):  # 전체 무당벌레 데이터 프레임
        if haversine(kellog.gps[i], df.gps[j], unit='km') <= 18 and kellog.Year[i] == df.Year[j]:
            kellog[df.Species[j]+'_18'][i]+=1

  0%|                                                                                          | 0/253 [00:00<?, ?it/s]<ipython-input-7-a2d82a62ea73>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kellog[df.Species[j]+'_18'][i]+=1
100%|████████████████████████████████████████████████████████████████████████████████| 253/253 [15:46<00:00,  3.74s/it]


In [ ]:
for i in tqdm(range(len(kellog))): # target만 뽑아놓은 데이터 프레임
    for j in range(len(df.gps)):  # 전체 무당벌레 데이터 프레임
        if haversine(kellog.gps[i], df.gps[j], unit='km') <= 18 and kellog.Year[i] == df.Year[j]:
            kellog[df.Species[j]+'_18'][i]+=1

### 1-2. 타겟종 주변의 새 정보 파악

In [8]:
# Ebird에서 가져온 새 목록 합치기
os.chdir(path = './Data/E_bird/')

# directory 내의 csv들을 전부 가져와서
for i in range(len(os.listdir())):
    globals()['bird{}'.format(i)] = pd.read_csv(os.listdir()[i],sep='\t')

# for_merge 라는 변수에 모두 합쳐놓음
for_merge = bird1.iloc[0:0]
for i in range(26):
    for_merge = pd.concat([for_merge,globals()['bird{}'.format(i)]])
bird2021 = pd.read_csv("../새2021.csv")
for_merge = pd.concat([for_merge, bird2021]).reset_index(drop=True)


# EBIRD자료는 GPS정확도를 제공하지 않으므로, GPS정확도가 존재한다면 INAT의 자료임
for_merge = for_merge[for_merge.coordinateUncertaintyInMeters.isnull()].reset_index(drop=True)

# 필요한 컬럼들만 남김
for_merge=for_merge[['species','year','month','decimalLatitude','decimalLongitude']].reset_index(drop=True)

# 종 이름이 NAN이면 쓸수 없으므로 DROP
for_merge=for_merge.drop(for_merge[for_merge.species.isnull()==True].index).reset_index(drop=True)

# haversine은 set에 들어있는 gps값을 받음 (lat, lon) 이형식으로 변환하는 과정
for_gps = []
for i in tqdm(range(len(for_merge))):
    for_gps.append( (for_merge['decimalLatitude'][i], for_merge['decimalLongitude'][i]) )
    
for_merge['gps'] = for_gps



for_merge=for_merge[(for_merge.species!='Locustella certhiola')&(for_merge.species!='Locustella ochotensis')&
         (for_merge.species!='Ianthocincla pectoralis')&(for_merge.species!='Luscinia calliope')&
         (for_merge.species!='Iduna aedon')&(for_merge.species!='Phylloscopus sibillatrix')&
         (for_merge.species!='Muscicapa latirostris')&(for_merge.species!='Tiaris bicolor')&
         (for_merge.species!='Tiaris canorus')&(for_merge.species!='Lonchura oryzivora')&
         (for_merge.species!='Lonchura cucullata')&(for_merge.species!='Estrilda coerulescens')&
         (for_merge.species!='alcarius ornatus')]

# 무당벌레는 관측지역에 잡힌 숫자를 믿을 수 없어서 관측 자체만 믿었음, 새 역시도 drop_duplicate로 동일한 절차 진행
for_merge = for_merge.drop_duplicates(keep='first').reset_index(drop=True)


# 타겟종의 데이터프레임에 주변 정보를 카운트해서 넣을 컬럼 생성
for i in for_merge.species.value_counts().index:
    kellog[i+'_18'] = 0

C:\Users\DAEGYEONGLEE\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (49) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\DAEGYEONGLEE\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (14,16,17,36,37,38,39,41,43,44,46,48) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
100%|█████████████████████████████████████████████████████████████████████| 3034469/3034469 [00:32<00:00, 92261.23it/s]


In [9]:
for i in tqdm(range(len(kellog))): # 타겟종만 뽑아놓은 데이터 프레임
    for j in range(len(for_merge.gps)):  # 전체 새 데이터 프레임
            if haversine(kellog.gps[i], for_merge.gps[j], unit='km') <= 18 and kellog.Year[i] == for_merge.year[j]:
                # 타겟종 관측 좌표와 새 관측좌표의 거리가 1km이하이고, 해당 좌표의 관측 연도가 같다면 count +1
                kellog[for_merge.species[j]+'_18'][i]+=1


  0%|                                                                                          | 0/253 [00:00<?, ?it/s]<ipython-input-9-5a383200347b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kellog[for_merge.species[j]+'_18'][i]+=1
100%|██████████████████████████████████████████████████████████████████████████████| 253/253 [2:15:00<00:00, 32.02s/it]


In [12]:
kellog.to_csv("./kellog_absence_주변종.csv")